<a href="https://colab.research.google.com/github/ucl-casa-ce/casa0018/blob/main/Week6/CASA0018_6_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning 
To demonstrate the process of transfer learning we will explore an example based on VGG16. 

The VGG16 model was developed by the Visual Graphics Group (VGG) at Oxford and was described in the 2014 paper titled [“Very Deep Convolutional Networks for Large-Scale Image Recognition.”](https://arxiv.org/abs/1409.1556)

By default, the model expects color input images to be rescaled to the size of 224×224 pixels.

The pre-trained model can be loaded as follows. Running the code block will load the VGG16 model and download the model weights. Notice the last few Dense layers and the total number of trainable parameters.

In [ ]:
# example of loading the vgg16 model
from keras.applications.vgg16 import VGG16
# load model
model = VGG16()
# summarize the model
model.summary()

## Exercise
You could try the same for other models such as InceptionV3 or ResNet50. You just need to edit the lines below to explore those models. Take a look at the API documentation to see how the models are referenced - note that InceptionV3 may not be labelled quite how you first imagine and watch out for case sensitivity. 

https://keras.io/api/applications/resnet/

https://keras.io/api/applications/inceptionv3/

In [ ]:
# example of loading the resnet50 model
from keras.applications.####ADD_IN_APPLICATION_NAME### import ResNet50
# load model
model = ###ADD_IN_MODEL_NAME###()
# summarize the model
model.summary()

In [ ]:
# example of loading the InceptionV3 model
from keras.applications.####ADD_IN_APPLICATION_NAME### import InceptionV3
# load model
model = ###ADD_IN_MODEL_NAME###()
# summarize the model
model.summary()

# Examples of how to use the pre-trained models 

## Simple classifer - no transfer learning.
There is no transfer learning here but as a reminder, we can use this pre-trained model to classify an image.

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

# load an image from file
image = load_img('D_Arlo.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# load the model
model = VGG16()
# predict the probability across all output classes
p = model.predict(image)
# convert the probabilities to class labels
lst = decode_predictions(p, top=3)
print(lst)

for label in lst[0]:
  # print(label)
  print('I think this is a %s and I am %.2f%% confident.' % (label[1], label[2]*100))

## Adding Transfer Learning

### First we will grab some data that we want to be able to classify.

In [ ]:
!wget --no-check-certificate \
    https://github.com/ucl-casa-ce/casa0018/blob/main/Week6/data/validating.zip?raw=true \
    -O /content/validating.zip
  
!wget --no-check-certificate \
    https://github.com/ucl-casa-ce/casa0018/blob/main/Week6/data/training.zip?raw=true \
    -O /content/training.zip

Copy the data across to the content folder in the Colab Notebook.

In [ ]:
import os
import zipfile

local_zip = '/content/training.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

local_zip = '/content/validating.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

Show some info on the images:

In [ ]:
mugs_dir = os.path.join('/content/training/mugs')
plant_dir = os.path.join('/content/training/plant')
unknown_dir = os.path.join('/content/training/unknown')

print('total training mugs images:', len(os.listdir(mugs_dir)))
print('total training plants images:', len(os.listdir(plant_dir)))
print('total training unknown images:', len(os.listdir(unknown_dir)))

mugs_files = os.listdir(mugs_dir)
print(mugs_files[:10])

plant_files = os.listdir(plant_dir)
print(plant_files[:10])

unknown_files = os.listdir(unknown_dir)
print(unknown_files[:10])

Display some of the images

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_mugs = [os.path.join(mugs_dir, fname) 
                for fname in mugs_files[0:pic_index]]
next_plant = [os.path.join(plant_dir, fname) 
                for fname in plant_files[0:pic_index]]
next_unknown = [os.path.join(unknown_dir, fname) 
                for fname in unknown_files[0:pic_index]]

for i, img_path in enumerate(next_mugs+next_plant+next_unknown):
  #print(img_path)
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

### Use transfer learning to create new model

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras.applications.vgg16 import VGG16
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

# Note: ImageDataGenerator in Keras implements "in-place" augementation - ie it replaces
# the batch of images past in with a transformed set. 
# This means that when the network is trained, each Epoch sees new variations of our images
# https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/ 

TRAINING_DIR = "/content/training/"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "/content/validating/"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

label_map = (train_generator.class_indices)
print("Classes found in data are: ", label_map)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

Next up we use a pretrained model as a starting point to build our new model. 

In [ ]:
# defining a variable to hold image since since it is called in few places below
IMAGE_SIZE = [150, 150]

# load model without classifier layers

pretrained_model = VGG16(weights='imagenet', 
                         include_top=False ,
                         input_shape=[*IMAGE_SIZE, 3])
#pretrained_model = MobileNetV2(input_shape=[*IMAGE_SIZE, 3], 
#                               include_top=False)
#pretrained_model = ResNet50(weights='imagenet', 
#                            include_top=False, 
#                            input_shape=[*IMAGE_SIZE, 3])
#pretrained_model = MobileNet(weights='imagenet', 
#                             include_top=False, 
#                             input_shape=[*IMAGE_SIZE, 3])

pretrained_model.trainable = False

model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Finaly we can run our training:

In [ ]:
# Run training and save to history so that we can plot metrics
history = model.fit(train_generator, 
                    epochs=20, 
                    validation_data = validation_generator, 
                    verbose = 1)

# we can save the model so that we can reload at a later date
# It appears in the content folder - download it before your session ends
# https://www.tensorflow.org/guide/keras/save_and_serialize
model.save("mugsplant.h5")

Once training is complete we can plot results - this should be starting to look familiar. 

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc=0)

plt.subplot(1, 2, 2)

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc=0)

plt.tight_layout()


# Testing your model
To see how well this model has learnt you can upload an image and classify it. Some sample images have been included in the github repo in a folder called testing. You can drag and drop those into the Contents folder on left panel. 

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

# load an image from file
img = load_img('1.jpg', target_size=(150, 150))

plt.imshow(img)
plt.show()

# convert to an array that we can use as input to our model
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

# run prediction
p = model.predict(images)
  
# create a look up of the class labels generated from the 
# "flow_from_directory" method used in the generator and print result
predicted_class_indices = np.argmax(model.predict(x), axis=-1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

print('I think the image above is "%s" and I am %.2f%% confident.' % (predictions[0], p[0][predicted_class_indices]*100))

print(p[0])
